In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


# Advanced Construction Item Description Standardization

## Overview
This notebook implements an advanced standardization system for construction item descriptions using BERT-based similarity detection, comprehensive pattern matching, and intelligent duplicate detection.

## Key Features
- BERT embeddings for semantic similarity
- Fuzzy matching for near-duplicates
- Advanced clustering for item grouping
- Comprehensive construction terminology standardization

## Implementation
The process consists of three main components:
1. Library imports and setup
2. Advanced standardizer class implementation
3. Data processing and results generation

In [ ]:
%pip install sentence-transformers scikit-learn rapidfuzz transformers torch

## Import Libraries
Import required libraries for text processing, machine learning, and data manipulation. This includes specialized libraries for BERT embeddings and fuzzy matching.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import re
import logging
from typing import Tuple, List, Dict
from fractions import Fraction
from rapidfuzz import fuzz
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## Standardizer Class Implementation
Define the advanced standardizer class with comprehensive pattern matching and similarity detection capabilities.

In [ ]:
class ConstructionItemStandardizer:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self._initialize_patterns()
        
    def _initialize_patterns(self):
        """Initialize all standardization patterns"""
        # Standard measurements
        self.measurement_patterns = {
            # Fractions and Mixed Numbers
            r'(\d+)\s*/\s*(\d+)': r'\1/\2',
            r'(\d+)-(\d+)/(\d+)': r'\1-\2/\3',
            r'(\d+)\s+(\d+)/(\d+)': r'\1-\2/\3',
            
            # Dimensions with various separators
            r'(\d+(?:\.\d+)?)\s*(?:"|IN|INCH|INCHES)\s*[xX×]\s*(\d+(?:\.\d+)?)\s*(?:"|IN|INCH|INCHES)\s*[xX]\s*(\d+(?:\.\d+)?)\s*(?:"|IN|INCH|INCHES)': r'\1IN X \2IN X \3IN',
            r'(\d+(?:\.\d+)?)\s*(?:"|IN|INCH|INCHES)\s*[xX]\s*(\d+(?:\.\d+)?)\s*(?:"|IN|INCH|INCHES)': r'\1IN X \2IN',
            
            # Decimal measurements
            r'(\d+\.\d+)(?:"|IN|INCH|INCHES)\b': r'\1IN',
            r'(\d+\.\d+)(?:\'|FT|FEET|FOOT)\b': r'\1FT',
            
            # Ranges and sizes
            r'(\d+)(?:\s*-\s*TO\s*-\s*)(\d+)': r'\1-\2',
            r'SIZE\s*:?\s*(\d+)': r'\1',
            r'#(\d+)': r'NO.\1',
        }
        
        # Expanded unit standardization
        self.unit_standardization = {
            # Length/Distance
            r'\bINCHES\b|\bINCH\b|\bIN\b|"': 'IN',
            r'\bFEET\b|\bFOOT\b|\bFT\b|\'': 'FT',
            r'\bYARDS?\b|\bYD\b': 'YD',
            r'\bMILLIMETERS?\b|\bMM\b': 'MM',
            r'\bCENTIMETERS?\b|\bCM\b': 'CM',
            r'\bMETERS?\b|\bM\b': 'M',
            
            # Weight
            r'\bPOUNDS?\b|\bLBS?\b': 'LB',
            r'\bOUNCES?\b|\bOZ\b': 'OZ',
            r'\bKILOGRAMS?\b|\bKG\b': 'KG',
            r'\bGRAMS?\b|\bG\b': 'G',
            
            # Volume
            r'\bGALLONS?\b|\bGAL\b': 'GAL',
            r'\bQUARTS?\b|\bQT\b': 'QT',
            r'\bPINTS?\b|\bPT\b': 'PT',
            r'\bLITERS?\b|\bL\b': 'L',
            r'\bFLUID\s+OUNCES?\b|\bFL\s*OZ\b': 'FL OZ',
            
            # Area
            r'\bSQUARE\s*FEET\b|\bSQ\s*FT\b': 'SQ FT',
            r'\bSQUARE\s*YARDS?\b|\bSQ\s*YD\b': 'SQ YD',
            r'\bSQUARE\s*METERS?\b|\bSQ\s*M\b': 'SQ M',
            
            # Quantity
            r'\bCOUNT\b|\bCT\b|\bPC?S?\b|\bPIECES?\b': 'PC',
            r'\bPACKS?\b|\bPK\b': 'PK',
            r'\bBOXES?\b|\bBX\b': 'BX',
            r'\bSETS?\b': 'SET',
            r'\bPAIRS?\b|\bPR\b': 'PR',
            
            # Electrical
            r'\bVOLTS?\b|\bV\b': 'V',
            r'\bAMPS?\b|\bA\b': 'A',
            r'\bWATTS?\b|\bW\b': 'W',
        }
        
        # Construction specific terms
        self.construction_terms = {
            # Materials
            r'\bSTAINLESS\s+STEEL\b': 'SS',
            r'\bCARBON\s+STEEL\b': 'CS',
            r'\bHIGH\s+DENSITY\b': 'HD',
            r'\bLOW\s+DENSITY\b': 'LD',
            r'\bPRESSURE\s+TREATED\b': 'PT',
            r'\bULTRA\s+HIGH\s+MOLECULAR\s+WEIGHT\b': 'UHMW',
            r'\bPOLYVINYL\s+CHLORIDE\b': 'PVC',
            r'\bABS\s+PLASTIC\b': 'ABS',
            
            # Finishes
            r'\bPOWDER\s+COATED\b': 'PWD CTD',
            r'\bGALVANIZED\b': 'GALV',
            r'\bELECTRO\s+GALVANIZED\b': 'E-GALV',
            r'\bHOT\s+DIPPED\s+GALVANIZED\b': 'HDG',
            r'\bBRUSHED\s+NICKEL\b': 'BN',
            r'\bOIL\s+RUBBED\s+BRONZE\b': 'ORB',
            
            # Hardware
            r'\bCOUNTERSUNK\b': 'CSK',
            r'\bPHILLIPS\s+HEAD\b': 'PH',
            r'\bFLAT\s+HEAD\b': 'FH',
            r'\bHEX\s+HEAD\b': 'HH',
            r'\bWASHER\s+HEAD\b': 'WH',
            r'\bBUTTON\s+HEAD\b': 'BH',
            
            # Common descriptors
            r'\bADJUSTABLE\b': 'ADJ',
            r'\bHEAVY\s+DUTY\b': 'HD',
            r'\bLIGHT\s+DUTY\b': 'LD',
            r'\bCOMMERCIAL\s+GRADE\b': 'COM GR',
            r'\bINDUSTRIAL\s+GRADE\b': 'IND GR',
            r'\bPROFESSIONAL\s+GRADE\b': 'PRO GR',
            r'\bWEATHER\s+RESISTANT\b': 'WR',
            r'\bWATER\s+RESISTANT\b': 'WR',
            r'\bUNIVERSAL\b': 'UNI',
            r'\bSTANDARD\b': 'STD',
        }
        
        # Common color codes
        self.color_codes = {
            r'\bWHITE\b': 'WHT',
            r'\bBLACK\b': 'BLK',
            r'\bBROWN\b': 'BRN',
            r'\bGRAY\b|\bGREY\b': 'GRY',
            r'\bBEIGE\b': 'BGE',
            r'\bBRONZE\b': 'BRZ',
            r'\bCLEAR\b': 'CLR',
            r'\bNATURAL\b': 'NAT',
        }
        
        # Updated brands list (expandable)
        self.brands = [
            'PURDY', 'USG', 'QUIKRETE', 'GORILLA', 'MINWAX', 'DEWALT',
            'MILWAUKEE', 'MAKITA', 'BOSCH', 'RYOBI', 'WERNER', '3M',
            'SIMPSON', 'HUSKY', 'RIDGID', 'PROJECT SOURCE', 'EZ ANCOR',
            'STANLEY', 'IRWIN', 'CRAFTSMAN', 'KLEIN', 'CHANNELLOCK',
            'DREMEL', 'ESTWING', 'FESTOOL', 'GRACO', 'HILTI', 'KNIPEX',
            'LENOX', 'METABO', 'NORTON', 'PORTER-CABLE', 'SENCO', 
            'TEKTON', 'WAGNER', 'ZIRCON'
        ]

    def standardize_measurements(self, text: str) -> str:
        """Apply measurement standardization patterns"""
        for pattern, replacement in self.measurement_patterns.items():
            text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
        return text

    def standardize_units(self, text: str) -> str:
        """Standardize unit representations"""
        for pattern, replacement in self.unit_standardization.items():
            text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
        return text

    def standardize_construction_terms(self, text: str) -> str:
        """Standardize common construction terms"""
        for pattern, replacement in self.construction_terms.items():
            text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
        return text

    def extract_sku(self, text: str) -> Tuple[str, str]:
        """Extract SKU number from the beginning of description"""
        if pd.isna(text):
            return '', ''
        
        sku_match = re.match(r'^(\d+)\s+(.*)', str(text))
        if sku_match:
            return sku_match.group(1), sku_match.group(2)
        return '', text

    def clean_description(self, text: str) -> str:
        """Main cleaning function for item descriptions"""
        if pd.isna(text):
            return ''
        
        # Extract SKU
        sku, desc = self.extract_sku(text)
        
        # Convert to uppercase and clean
        desc = desc.upper()
        desc = self.standardize_measurements(desc)
        desc = self.standardize_units(desc)
        desc = self.standardize_construction_terms(desc)
        
        # Remove special characters and fix spacing
        desc = re.sub(r'[^\w\s/.\-X]', '', desc)
        desc = re.sub(r'\s+', ' ', desc).strip()
        
        # Move brand to start if present
        for brand in self.brands:
            if brand in desc.upper():
                desc = re.sub(rf'{brand}', '', desc, flags=re.IGNORECASE)
                desc = f"{brand} {desc.strip()}"
                break
        
        return desc.strip()

    def process_dataframe(self, df: pd.DataFrame, item_desc_column: str) -> pd.DataFrame:
        """Process the entire dataframe and add analysis columns"""
        self.logger.info("Starting data processing...")
        
        # Verify column exists
        if item_desc_column not in df.columns:
            raise ValueError(f"Column '{item_desc_column}' not found in dataframe. Available columns: {df.columns.tolist()}")
        
        # Create copy to avoid modifying original
        df_processed = df.copy()
        
        # Clean descriptions
        self.logger.info("Cleaning descriptions...")
        df_processed['Cleaned_Description'] = df_processed[item_desc_column].apply(self.clean_description)
        
        # Extract SKUs
        self.logger.info("Extracting SKUs...")
        df_processed['SKU'] = df_processed[item_desc_column].apply(lambda x: self.extract_sku(x)[0])
        
        # Add analysis columns
        self.logger.info("Adding analysis columns...")
        df_processed['Duplicate_Group'] = df_processed.groupby('Cleaned_Description').ngroup()
        df_processed['Is_Duplicate'] = df_processed.duplicated(subset='Cleaned_Description', keep=False)
        
        return df_processed

print("ConstructionItemStandardizer class defined successfully!")

## Process Data
Execute the standardization process on the input data and generate results.

In [ ]:
# Initialize standardizer
standardizer = ConstructionItemStandardizer()

try:
    # Read data from Lakehouse
    df = spark.table("inscope_items_output").toPandas()
    
    # Process data
    df_processed = standardizer.process_dataframe(df, item_desc_column='item_description')
    
    # 1. Create table without duplicates (unique items)
    df_unique = df_processed.drop_duplicates(subset='Cleaned_Description', keep='first')
    spark_df_unique = spark.createDataFrame(df_unique)
    
    # 2. Create table with only duplicates
    df_duplicates = df_processed[df_processed['Is_Duplicate']].sort_values('Duplicate_Group')
    spark_df_duplicates = spark.createDataFrame(df_duplicates)
    
    # Save both tables to Lakehouse
    spark_df_unique.write.mode("overwrite").saveAsTable("inscope_items_unique")
    spark_df_duplicates.write.mode("overwrite").saveAsTable("inscope_items_duplicates")
    
    # Print summary statistics
    print("Summary Statistics:")
    print("-" * 50)
    print(f"Total original items: {len(df_processed):,}")
    print(f"Unique items: {len(df_unique):,}")
    print(f"Items with duplicates: {len(df_duplicates):,}")
    print(f"Number of duplicate groups: {df_duplicates['Duplicate_Group'].nunique():,}")
    
    # Display samples of both tables
    print("\n1. Sample of Unique Items (no duplicates):")
    print("-" * 50)
    display(spark_df_unique.limit(5))
    
    print("\n2. Sample of Duplicate Items:")
    print("-" * 50)
    display(spark_df_duplicates.limit(5))
    
    # Additional analysis of duplicate groups
    print("\nDuplicate Groups Analysis:")
    print("-" * 50)
    duplicate_groups = df_duplicates.groupby('Duplicate_Group').agg({
        'item_description': 'count',
        'Cleaned_Description': 'first'
    }).reset_index()
    duplicate_groups.columns = ['Group_ID', 'Count', 'Standardized_Description']
    duplicate_groups = duplicate_groups.sort_values('Count', ascending=False)
    
    print("\nTop 5 Most Common Duplicates:")
    display(spark.createDataFrame(duplicate_groups.head()))
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

## Output Tables
The process generates two main output tables:
1. `standardized_items_advanced`: Complete standardized dataset
2. `duplicate_groups`: Groups of similar items for review

## Usage Notes
- Run cells in sequence
- Review the sample outputs for verification
- Check logging output for process status
- Adjust similarity thresholds if needed

## Maintenance
To update standardization patterns:
1. Modify relevant pattern dictionaries in the standardizer class
2. Re-run the notebook to apply changes
3. Review results for accuracy